# Using the k-Nearest Neighbours Algorithm to Predict Bike Availability

The machine learning algorithm we decided to use in our project to predict bike availability was a tweaked version of the k-nearest neighbours algorithm. We have chosen this algorithm both for it's simplicity and, we would argue, it's suitability with some alteration for the task of predicting bike availability based on weather data. Our primary source of information for learning about this algorithm was the chapter 8.2 in Tom Mitchel's book "Machine Learning".

In the k-nearest neighbours algorithm, the instance we are trying to predict what a particular instance is based on the k (some number) "nearest neighbours" from out data set, that is to say the instances from our dataset that our closest to the instance being predicted in it's known qualities. This algorithm is usually used to allocate instances into discrete categories, in which cases the instance will be given the same classification as the majority of it's k-nearest neighbours. As the predictions we are assigning are continuous, we will use the average of the k-nearest neighbours, i.e. the average available bikes of the nearest neighbours selected based on similarity of weather conditions. To maximise accuracy, we will be implementing a version of k nearest neighbours outlined by Mitchell on pp.233-4, distance weighted nearest neighbours where k is all training examples. In terms of our project, this means our prediction will be based on the weighted average of all of the previous bike availability data for a station, with examples having a higher weight in the average the closer they are in weather to weather for our prediction.  

In [16]:
import mysql.connector
import math
import time
from datetime import datetime, timedelta
import requests
from random import randint
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

warnings.filterwarnings('ignore')

### Prediction Model

In [31]:
def nearest_neighbours(station_no):
    data = requests.get("http://api.openweathermap.org/data/2.5/forecast?q=Dublin&appid=6fb76ecce41a85161d4c6ea5e2758f2b").json()
    mydb = mysql.connector.connect(
        host="newdublinbikesinstance.cevl8km57x9m.us-east-1.rds.amazonaws.com",
        user="root",
        passwd="secretpass"
    )

    cursor = mydb.cursor(buffered=True)

    counter = 0
    
    predictions = []

    for forecast in data['list']: #retrieving the data and time information from the api call to display
        dt = forecast['dt']
        dt = int(dt)
        dt = datetime.utcfromtimestamp(dt).strftime('%Y-%m-%d %H:%M:%S')
        date, time = dt.split(" ")

        #for time in dates_and_times[date]:

        forecast_key =  date + " " + time

        prediction_date = datetime.strptime(date, '%Y-%m-%d')

        prediction_day = prediction_date.weekday()

        dt_time = datetime.strptime(time, '%H:%M:%S')

        prediction_time = timedelta(hours=dt_time.hour, minutes=dt_time.minute, seconds=dt_time.second)

        prediction_temp = forecast['main']['temp']

        prediction_weather = forecast['weather'][0]['main']
        
        weight_total = 0 
        weighted_predictors_total = 0 
        
        #now that we have a prediction for weather for that particular time and date, we can compare it to our previous records
        
        #if the prediction is on a weekday, we only use weekday records
        if (prediction_day < 5):                                                                                                                                                                                                                                                                                                                                                                            #limited for scalability                     
            cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 15 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' AND weather.description = '%s' AND weekday(weather.date) < 5 LIMIT 50000" % (station_no, prediction_weather))
            #for the station number in question we are retrieving all of our records where the general weather description is the same (raining, clouds, etc.) and the time of day is roughly the same, that is to say less than 11 minutes off. We will not be lookng at records where the station was not open.
            rows = cursor.fetchall()
            if len(rows) == 0: #if a currently unknown weather is encountered (one there is not previous data on), we will do the same as above but for all weather description types, i.e. if snow is encountered for the first time we will take records with rainy, clear, clouds, mist, drizzle and any other weather types
                cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 15 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' weekday(weather.date) < 5 LIMIT 50000" % station_no)
                rows = cursor.fetchall()
        #if weekend, use only records from that day:
        else:
            cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 15 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' AND weather.description = '%s' AND weekday(weather.date) = %s LIMIT 50000" % (station_no, prediction_weather, prediction_day))
            rows = cursor.fetchall()
            if len(rows) == 0: #if a currently unknown weather is encountered (one there is not previous data on), we will do the same as above but for all weather description types, i.e. if snow is encountered for the first time we will take records with rainy, clear, clouds, mist, drizzle and any other weather types
                cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 15 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' AND weekday(weather.date) = %s LIMIT 50000" % station_no, prediction_day)
                rows = cursor.fetchall()
        #we will now get the weighted average of all the records retrieved above   
        for row in rows:
            row_temp = row[6]
            row_bikes = row[2]
            row_time = row[8]
            temp_weight = 1/(math.sqrt((row_temp - prediction_temp)**2) + 0.5) #the difference between the temperature in a record and the predicted temperature
            time_weight = 1/(math.sqrt((round((row_time - prediction_time).total_seconds()/60)**2)) + 0.5) #the difference between the time in a record and the time of the prediction
            weight = temp_weight + time_weight #weight is determined based on the difference in both time and temperature
            #adding the weight and weighted predictions from this record to the totals
            weight_total += weight   
            weighted_predictor = row_bikes * weight
            weighted_predictors_total += weighted_predictor
        #finally, our prediction is the waited average available bikes from the records we retrieved
        prediction = round(weighted_predictors_total/weight_total)
        #print(prediction)
        predictions.append(prediction)
        counter += 1
        if counter == 9:
            break

    return predictions
        

In [33]:
prediction = nearest_neighbours(2)

In [34]:
print("Available bikes prediction for station 2 is:", prediction)

Available bikes prediction for station 2 is: [6, 6, 5, 2, 4, 4, 4, 4, 4]


We will now use the time magic command to evaluate the speed of the algorithm:

In [10]:
#getting all station numbers

data = requests.get("http://api.openweathermap.org/data/2.5/forecast?q=Dublin&appid=6fb76ecce41a85161d4c6ea5e2758f2b").json()
    
mydb = mysql.connector.connect(
    host="newdublinbikesinstance.cevl8km57x9m.us-east-1.rds.amazonaws.com",
    user="root",
    passwd="secretpass"
    )

cursor = mydb.cursor(buffered=True)
    
cursor.execute("SELECT DISTINCT station_no FROM innodb.station_var")
                
stations = cursor.fetchall()

station_nos = []

for i in stations:
    station_nos.append(i[0])

In [20]:
times = []
max_index = len(station_nos) - 1
executions = 0
while executions < 50:
    station_no = station_nos[randint(0, max_index)]
    start = time.time()
    #timed section begins
    nearest_neighbours(station_no)
    #timed section ends
    stop = time.time()
    times.append(stop-start)
    executions += 1

total = 0
for i in times:
    total += i

print("Average time over", executions, "executions is:", total/executions)

Average time over 50 executions is: 14.351784052848815


## Accuracy Evaluation

The method below evaluates the average accuracy of the predictive model for a given number of random tests using past data:

In [204]:
mydb = mysql.connector.connect(
        host="newdublinbikesinstance.cevl8km57x9m.us-east-1.rds.amazonaws.com",
        user="root",
        passwd="secretpass"
    )

cursor = mydb.cursor(buffered=True)

In [205]:
cursor.execute("SELECT * FROM innodb.weather")

#cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather ON station_var.last_update_date = weather.date WHERE (minute(timediff(station_var.lat_update_time, weather.time)) < 1 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) AND station_var.status = 'OPEN' LIMIT 100;")

#joined_rows = cursor.fetchall()

#df = pd.DataFrame(joined_rows)

weather_rows = cursor.fetchall()

weather_df = pd.DataFrame(weather_rows)

print(weather_df.head())

print("----------------------------")

print(weather_df.shape)

        0     1        2           3
0  280.73  Mist 09:13:51  2019-02-28
1  283.97  Rain 15:33:51  2019-03-02
2  282.31  Rain 16:33:48  2019-03-02
3  282.15  Rain 17:13:37  2019-03-02
4  280.56  Rain 19:16:32  2019-03-02
----------------------------
(2034, 4)


In [206]:
weather_df.columns = ['temp', 'weather', 'time', 'date']
print(weather_df.head())

     temp weather     time        date
0  280.73    Mist 09:13:51  2019-02-28
1  283.97    Rain 15:33:51  2019-03-02
2  282.31    Rain 16:33:48  2019-03-02
3  282.15    Rain 17:13:37  2019-03-02
4  280.56    Rain 19:16:32  2019-03-02


In [217]:
cursor.execute("SELECT * FROM innodb.station_var")
station_rows = cursor.fetchall()
station_df = pd.DataFrame(station_rows)
print(station_df.head())
print("--------------------")
print(station_df.shape)

      0   1   2           3        4    5
0  OPEN  30   0  2019-02-28 09:18:03   42
1  OPEN  18   2  2019-02-28 09:14:48   30
2  OPEN  18  15  2019-02-28 09:20:31   54
3  OPEN  38   2  2019-02-28 09:20:40  108
4  OPEN   1  38  2019-02-28 09:20:12   56
--------------------
(1020092, 6)


In [218]:
station_df.columns = ['status', 'stands', 'bikes', 'date', 'station_time', 'station_no']
print(station_df.head())

  status  stands  bikes        date station_time  station_no
0   OPEN      30      0  2019-02-28     09:18:03          42
1   OPEN      18      2  2019-02-28     09:14:48          30
2   OPEN      18     15  2019-02-28     09:20:31          54
3   OPEN      38      2  2019-02-28     09:20:40         108
4   OPEN       1     38  2019-02-28     09:20:12          56


In [219]:
station_df = station_df[(station_df.status == 'OPEN')]
station_df = station_df.drop("status", axis=1)
station_df = station_df.drop("stands", axis=1)
merged_df = weather_df.merge(station_df, on='date')
print(merged_df.head())
print(merged_df.shape[0])

     temp weather     time        date  bikes station_time  station_no
0  280.73    Mist 09:13:51  2019-02-28      0     09:18:03          42
1  280.73    Mist 09:13:51  2019-02-28      2     09:14:48          30
2  280.73    Mist 09:13:51  2019-02-28     15     09:20:31          54
3  280.73    Mist 09:13:51  2019-02-28      2     09:20:40         108
4  280.73    Mist 09:13:51  2019-02-28     38     09:20:12          56
50283865


In [220]:
df = merged_df[(abs(merged_df.time - merged_df.station_time) < pd.to_timedelta('00:15:00'))]
df = df.drop_duplicates()
print(df[(df.station_no == 42)].tail())
print(df.shape[0])

            temp weather     time        date  bikes station_time  station_no
50278328  276.21  Clouds 01:32:52  2019-04-17     30     01:43:57          42
50279232  277.35  Clouds 04:48:42  2019-04-17     29     04:49:57          42
50280136  279.99  Clouds 07:48:57  2019-04-17      0     07:53:14          42
50281040  284.02  Clouds 10:46:38  2019-04-17      0     10:53:38          42
50283752  281.68    Mist 19:47:41  2019-04-17     22     19:51:33          42
611594


In [336]:
def check_accuracy(no_tests, df):

    train_df, test_df = train_test_split(df, test_size=0.3)
    
    train_df['date'] = pd.to_datetime(train_df['date'])
        
    tests_completed = 0
    
    #total_prediction_error = 0
    
    #joined_rows_count = len(joined_rows)
    
    no_test_rows = test_df.shape[0]
    
    prediction_list = []
    
    true_list = []
    
    while tests_completed < no_tests: #repeat below for the given amount of tests to complete
        
        #station_no = station_no_rows[random.randint(0, station_no_rows_count - 1)][0] #select a random station number
        
        #while test_row[3] == date:
        #test_row = test_df.sample() #select a random weather record
        
        #print(test_row)
        
        #return
        
        random_row = randint(0, no_test_rows - 1)

        temp = test_df.iloc[random_row]['temp'] #storing the data from that weather record

        weather = test_df.iloc[random_row]['weather'] 
        
        time = test_df.iloc[random_row]['time']
        
        date = test_df.iloc[random_row]['date']
        
        day = date.weekday()
        
        bikes_available = test_df.iloc[random_row]['bikes']
        
        true_list.append(bikes_available)
        
        station_no = test_df.iloc[random_row]['station_no']
        
        '''
        print("Time:", time)
        print("Temp:", temp)
        print("Weather:", weather)
        print("Date:", date)
        print("Day:", day)
        print("Station:", station_no)
        print("Bikes:", bikes_available)
        
        '''
        weight_total = 1 
        weighted_predictors_total = 1 
        if (day < 5):   
            train_rows = train_df[(train_df['weather'] == weather) & (train_df['station_no'] == station_no) & (abs(train_df['time'] - time) < pd.to_timedelta('00:15:00')) & (train_df['date'].dt.weekday < 5)]
            if train_rows.empty:  
                train_rows = train_df[(train_df['station_no'] == station_no) & (abs(train_df['time'] - time) < pd.to_timedelta('00:15:00')) & (train_df['date'].dt.weekday < 5)]
        else:
            train_rows = train_df[(train_df['weather'] == weather) & (train_df['station_no'] == station_no) & (abs(train_df['time'] - time) < pd.to_timedelta('00:15:00')) & (train_df['date'].dt.weekday == day)]
            if train_rows.empty:  
                train_rows = train_df[(train_df['station_no'] == station_no) & (abs(train_df['time'] - time) < pd.to_timedelta('00:15:00')) & (train_df['date'].dt.weekday == day)]
            #print(train_rows.head())
        for i in range(train_rows.shape[0] - 1):
            #print("hi")
            row_temp = train_rows.iloc[i]['temp']
            row_bikes = train_rows.iloc[i]['bikes']
            row_time = train_rows.iloc[i]['time']
            #print(row_temp)
            #print(row_bikes)
            #print(row_time)
            #print()
            #print(round((row_time - time).total_seconds()))
            #return
            temp_weight = 1/(abs(row_temp - temp) + 0.5)
            time_weight = 1/((abs(row_time - time).total_seconds()/60) + 0.5)
            weight = temp_weight + time_weight 
            weight_total += weight
            #print(weight_total)
            weighted_predictor = row_bikes * weight
            weighted_predictors_total += weighted_predictor
        print("----------------------TEST " + str(tests_completed + 1) + "----------------------")
        print("Station no:", station_no)
        if weight_total != 0:
            prediction = round(weighted_predictors_total/weight_total)
            prediction_list.append(prediction)
            print("Predicted bikes available:", prediction_list[tests_completed])
            print("Bikes actually available:", true_list[tests_completed])
            tests_completed += 1
        else:
            print("No suitable rows to make a prediction.")
            true_list.pop()
        print("--------------------------------------------------")
        print()
        
        
    print("------------------TESTS FINISHED------------------")
    return [true_list, prediction_list]

In [338]:
metrics = check_accuracy(5000, df)

----------------------TEST 1----------------------
Station no: 94
Predicted bikes available: 22.0
Bikes actually available: 26
--------------------------------------------------

----------------------TEST 2----------------------
Station no: 45
Predicted bikes available: 11.0
Bikes actually available: 16
--------------------------------------------------

----------------------TEST 3----------------------
Station no: 94
Predicted bikes available: 1.0
Bikes actually available: 1
--------------------------------------------------

----------------------TEST 4----------------------
Station no: 79
Predicted bikes available: 4.0
Bikes actually available: 2
--------------------------------------------------

----------------------TEST 5----------------------
Station no: 67
Predicted bikes available: 27.0
Bikes actually available: 23
--------------------------------------------------

----------------------TEST 6----------------------
Station no: 50
Predicted bikes available: 19.0
Bikes actua

----------------------TEST 49----------------------
Station no: 4
Predicted bikes available: 6.0
Bikes actually available: 4
--------------------------------------------------

----------------------TEST 50----------------------
Station no: 39
Predicted bikes available: 2.0
Bikes actually available: 5
--------------------------------------------------

----------------------TEST 51----------------------
Station no: 56
Predicted bikes available: 31.0
Bikes actually available: 30
--------------------------------------------------

----------------------TEST 52----------------------
Station no: 74
Predicted bikes available: 1.0
Bikes actually available: 0
--------------------------------------------------

----------------------TEST 53----------------------
Station no: 64
Predicted bikes available: 18.0
Bikes actually available: 19
--------------------------------------------------

----------------------TEST 54----------------------
Station no: 98
Predicted bikes available: 25.0
Bikes ac

----------------------TEST 95----------------------
Station no: 56
Predicted bikes available: 32.0
Bikes actually available: 32
--------------------------------------------------

----------------------TEST 96----------------------
Station no: 65
Predicted bikes available: 0.0
Bikes actually available: 0
--------------------------------------------------

----------------------TEST 97----------------------
Station no: 70
Predicted bikes available: 10.0
Bikes actually available: 10
--------------------------------------------------

----------------------TEST 98----------------------
Station no: 83
Predicted bikes available: 1.0
Bikes actually available: 1
--------------------------------------------------

----------------------TEST 99----------------------
Station no: 51
Predicted bikes available: 0.0
Bikes actually available: 0
--------------------------------------------------

----------------------TEST 100----------------------
Station no: 32
Predicted bikes available: 13.0
Bikes 

----------------------TEST 142----------------------
Station no: 106
Predicted bikes available: 6.0
Bikes actually available: 6
--------------------------------------------------

----------------------TEST 143----------------------
Station no: 40
Predicted bikes available: 4.0
Bikes actually available: 4
--------------------------------------------------

----------------------TEST 144----------------------
Station no: 79
Predicted bikes available: 13.0
Bikes actually available: 13
--------------------------------------------------

----------------------TEST 145----------------------
Station no: 6
Predicted bikes available: 13.0
Bikes actually available: 14
--------------------------------------------------

----------------------TEST 146----------------------
Station no: 87
Predicted bikes available: 18.0
Bikes actually available: 12
--------------------------------------------------

----------------------TEST 147----------------------
Station no: 64
Predicted bikes available: 16.0

----------------------TEST 189----------------------
Station no: 88
Predicted bikes available: 16.0
Bikes actually available: 17
--------------------------------------------------

----------------------TEST 190----------------------
Station no: 81
Predicted bikes available: 7.0
Bikes actually available: 15
--------------------------------------------------

----------------------TEST 191----------------------
Station no: 100
Predicted bikes available: 21.0
Bikes actually available: 25
--------------------------------------------------

----------------------TEST 192----------------------
Station no: 40
Predicted bikes available: 2.0
Bikes actually available: 0
--------------------------------------------------

----------------------TEST 193----------------------
Station no: 114
Predicted bikes available: 32.0
Bikes actually available: 35
--------------------------------------------------

----------------------TEST 194----------------------
Station no: 13
Predicted bikes available: 0

...

----------------------TEST 4767----------------------
Station no: 29
Predicted bikes available: 16.0
Bikes actually available: 18
--------------------------------------------------

----------------------TEST 4768----------------------
Station no: 31
Predicted bikes available: 8.0
Bikes actually available: 3
--------------------------------------------------

----------------------TEST 4769----------------------
Station no: 2
Predicted bikes available: 13.0
Bikes actually available: 17
--------------------------------------------------

----------------------TEST 4770----------------------
Station no: 41
Predicted bikes available: 1.0
Bikes actually available: 0
--------------------------------------------------

----------------------TEST 4771----------------------
Station no: 47
Predicted bikes available: 2.0
Bikes actually available: 2
--------------------------------------------------

----------------------TEST 4772----------------------
Station no: 11
Predicted bikes available: 1

----------------------TEST 4813----------------------
Station no: 45
Predicted bikes available: 11.0
Bikes actually available: 0
--------------------------------------------------

----------------------TEST 4814----------------------
Station no: 40
Predicted bikes available: 0.0
Bikes actually available: 0
--------------------------------------------------

----------------------TEST 4815----------------------
Station no: 56
Predicted bikes available: 14.0
Bikes actually available: 6
--------------------------------------------------

----------------------TEST 4816----------------------
Station no: 23
Predicted bikes available: 15.0
Bikes actually available: 18
--------------------------------------------------

----------------------TEST 4817----------------------
Station no: 74
Predicted bikes available: 24.0
Bikes actually available: 17
--------------------------------------------------

----------------------TEST 4818----------------------
Station no: 15
Predicted bikes available

----------------------TEST 4859----------------------
Station no: 56
Predicted bikes available: 11.0
Bikes actually available: 12
--------------------------------------------------

----------------------TEST 4860----------------------
Station no: 77
Predicted bikes available: 12.0
Bikes actually available: 5
--------------------------------------------------

----------------------TEST 4861----------------------
Station no: 19
Predicted bikes available: 13.0
Bikes actually available: 15
--------------------------------------------------

----------------------TEST 4862----------------------
Station no: 104
Predicted bikes available: 1.0
Bikes actually available: 1
--------------------------------------------------

----------------------TEST 4863----------------------
Station no: 23
Predicted bikes available: 12.0
Bikes actually available: 27
--------------------------------------------------

----------------------TEST 4864----------------------
Station no: 108
Predicted bikes availa

----------------------TEST 4905----------------------
Station no: 109
Predicted bikes available: 1.0
Bikes actually available: 0
--------------------------------------------------

----------------------TEST 4906----------------------
Station no: 11
Predicted bikes available: 11.0
Bikes actually available: 11
--------------------------------------------------

----------------------TEST 4907----------------------
Station no: 87
Predicted bikes available: 9.0
Bikes actually available: 11
--------------------------------------------------

----------------------TEST 4908----------------------
Station no: 60
Predicted bikes available: 4.0
Bikes actually available: 3
--------------------------------------------------

----------------------TEST 4909----------------------
Station no: 41
Predicted bikes available: 1.0
Bikes actually available: 1
--------------------------------------------------

----------------------TEST 4910----------------------
Station no: 73
Predicted bikes available: 

----------------------TEST 4953----------------------
Station no: 65
Predicted bikes available: 3.0
Bikes actually available: 4
--------------------------------------------------

----------------------TEST 4954----------------------
Station no: 105
Predicted bikes available: 2.0
Bikes actually available: 2
--------------------------------------------------

----------------------TEST 4955----------------------
Station no: 95
Predicted bikes available: 0.0
Bikes actually available: 2
--------------------------------------------------

----------------------TEST 4956----------------------
Station no: 90
Predicted bikes available: 0.0
Bikes actually available: 0
--------------------------------------------------

----------------------TEST 4957----------------------
Station no: 91
Predicted bikes available: 17.0
Bikes actually available: 19
--------------------------------------------------

----------------------TEST 4958----------------------
Station no: 86
Predicted bikes available: 2

In [339]:
true = metrics[0]
#print(true)
prediction = metrics[1]
#print(prediction)

In [341]:
#print("Accuracy score:", accuracy_score(true, prediction, normalize = False))
print("Normalized accuracy score:", accuracy_score(true, prediction))

Normalized accuracy score: 0.207
